In [3]:
#pip install pyarrow


     --------------------------------------- 24.3/24.3 MB 10.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#pip install awscli

     ---------------------------------------- 4.3/4.3 MB 6.2 MB/s eta 0:00:00
     ---------------------------------------- 79.8/79.8 kB 4.3 MB/s eta 0:00:00
     --------------------------------------- 11.2/11.2 MB 10.1 MB/s eta 0:00:00
     -------------------------------------- 548.2/548.2 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# import os
# !set AWS_SHARED_CREDENTIALS_FILE=awscli.ini
# path = "awscli.ini"
# os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path

# !aws configure list

      Name                    Value             Type    Location
      ----                    -----             ----    --------
   profile                <not set>             None    None
access_key     ****************ATZI shared-credentials-file    
secret_key     ****************e3kB shared-credentials-file    
    region                us-west-1      config-file    ~/.aws/config


In [15]:
%%time
import pyarrow.dataset as ds
dataset = ds.dataset("s3://ursa-labs-taxi-data/", partitioning=["year", "month"])
dataset

CPU times: total: 484 ms
Wall time: 21.3 s


In [16]:
# Question 1

In [27]:
%%time
import pyarrow.compute as pc
import pyarrow.parquet as pq
from datetime import datetime
count=0
total_amount = 0
columns_to_select = ['dropoff_at', 'total_amount']
for frag in dataset.get_fragments():
    frag_table = frag.to_table(columns = columns_to_select)
    sorted_indices = pc.sort_indices([dt.as_py().hour for dt in frag_table["dropoff_at"]])
    sorted_table = frag_table.take(sorted_indices)

    row_group_max_size = frag_table.to_batches()[0].num_rows
    pq.write_table(sorted_table, 'optimized_parquet_file.parquet', row_group_size=row_group_max_size)
    optimized_parquet_file = pq.ParquetFile('optimized_parquet_file.parquet')

    for i in range(optimized_parquet_file.num_row_groups):
        col_stats = optimized_parquet_file.metadata.row_group(i).column(0).statistics
        if col_stats.min.hour <= 14  and col_stats.max.hour >= 14:
            dropoffs= optimized_parquet_file.read_row_group(i)["dropoff_at"]
            filtered_timestamps = [14 <= timestamp.as_py().hour < 15 for timestamp in dropoffs]
            filtered_total_amount = pc.filter(optimized_parquet_file.read_row_group(i)["total_amount"], filtered_timestamps)
            count += len(filtered_total_amount)
            total_amount += pc.sum(filtered_total_amount).as_py()

if count>0:
    average = total_amount/count
    print("Average transaction between 2:00-2:59 PM: ",average)
else:
    print("No transaction between 2:00-2:59 PM")


Average transaction between 2:00-2:59 PM:  14.006634757751423
CPU times: total: 39min 58s
Wall time: 1h 13min 52s
